# Word2Vec model
> * Positive or Negative

In [4]:
import os
from glob import glob
import pandas as pd
import html
import itertools
import pickle
import re

from collections import namedtuple
from gensim.models import word2vec
import multiprocessing
from konlpy.tag import Mecab
from konlpy.utils import pprint
import numpy as np
from ckonlpy.tag import Twitter as ctwitter
mecab = Mecab()
ct = ctwitter()

C:\Users\pc\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [5]:
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
from sklearn.model_selection import train_test_split

# Word2Vec 생성

* vector size
 [ 1000, 2000 ]

In [6]:
rawdata = pd.read_csv('./data/sentiment_data/raw_data_for_sentiment.txt',header=None,encoding='utf-8')
print (rawdata.shape)

(491510, 2)


In [7]:
rawdata.head()

,0,1
0,이젠 민주개혁의 길로 사설 이 90년대가 우리 모두에게 성취의 시대이기를 기원하고...,1
1,민주당과 공화당은 새해부터 정계개편을 적극 추진한다는 방침이며 양당의 김영삼ㆍ김종필...,1
2,"4당,지자제 연합공천 대비 파트너 탐색전 이같은 구상을 실현시키기 위해 민정당은...",1
3,90년대는 국내적으로 정치민주화와 선진국에로의 도약여부가 판가름날 것이고 한반도를 ...,1
4,"특히 노태우대통령이 3일 대국민 특별담화를 발표, 지난시대 의 완전청산을 선언한 후...",1


In [8]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

In [9]:
import multiprocessing
cores = multiprocessing.cpu_count()
cores

12

In [10]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

In [12]:
d2v_docs = list()
for idx in tqdm(rawdata.index):
    text = rawdata.loc[idx,0]
    pos = mecab.pos(text)
    pos = [x[0] for x in pos ]#if (x[1] == u'VV' or x[1] == u'VA' or x[1] == u'NNB' or x[1] == u'NNP' or x[1] == u'NNG')]
    pos = [x for x in pos if not x in stopwords]
    label = rawdata.loc[idx,1]
    d2v_docs.append(TaggedDocument(pos, label))


  0%|          | 747/491510 [00:09<1:49:25, 74.75it/s]Exception in thread Thread-32:
Traceback (most recent call last):
  File "C:\Users\pc\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\pc\Anaconda3\lib\site-packages\tqdm\_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "C:\Users\pc\Anaconda3\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration
100%|██████████| 491510/491510 [1:01:09<00:00, 133.94it/s]


In [13]:
train, test = train_test_split(d2v_docs, test_size = 0.1)

In [33]:
pickle.dump(d2v_docs,open('./data/pre_data/tagged_data/pre_data_d2v_docs_for_word2vec_sentiment_by_mecab.pickled','wb'))
pickle.dump(train,open('./data/pre_data/train_test_Data/pre_data_train_for_word2vec_sentiment_by_mecab.pickled','wb'))
pickle.dump(test,open('./data/pre_data/train_test_Data/pre_data_test_for_word2vec_sentiment_by_mecab.pickled','wb'))

In [ ]:
train = pickle.load(open('./data/pre_data/train_test_Data/pre_data_train_for_word2vec_sentiment_by_mecab.pickled','rb'))
test = pickle.load(open('./data/pre_data/train_test_Data/pre_data_test_for_word2vec_sentiment_by_mecab.pickled','rb'))

In [14]:
x_train = [ x.words for x in tqdm(train)] 
y_train = [ x.tags for x in tqdm(train)] 
x_test = [ x.words for x in tqdm(test)] 
y_test = [ x.tags for x in tqdm(test)] 

100%|██████████| 49151/49151 [00:00<00:00, 1291507.09it/s]


In [15]:
w2v_model = word2vec.Word2Vec(sg=1, size=3000,window=8, workers = 6, iter = 20)
w2v_model.build_vocab(tqdm(x_train))
w2v_model.train(tqdm(x_train), total_examples=w2v_model.corpus_count, epochs=w2v_model.iter)

100%|██████████| 442359/442359 [20:31<00:00, 359.16it/s]


708667897

In [32]:
w2v_model.save('./model/w2v_model_size3000_window8_iter20_for_word2vec_sentiment_by_mecab.model')

In [29]:
w2v_model = word2vec.Word2Vec.load('./model/w2v_model_size3000_window8_iter20_for_word2vec_sentiment_by_mecab.model')

In [16]:
# importing bokeh library for interactive dataviz
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

# defining the chart
output_notebook()
plot_tfidf = bp.figure(plot_width=700, plot_height=600, title="A map of word vectors",
    tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
    x_axis_type=None, y_axis_type=None, min_border=1)

# getting a list of word vectors. limit to 10000. each is of 200 dimensions
word_vectors = [w2v_model[w] for w in list(w2v_model.wv.vocab.keys())[:10000]]

# dimensionality reduction. converting the vectors to 2d vectors
from sklearn.manifold import TSNE
tsne_model = TSNE(n_components=2, verbose=1, random_state=0)
tsne_w2v = tsne_model.fit_transform(word_vectors)
# putting everything in a dataframe
tsne_df = pd.DataFrame(tsne_w2v, columns=['x', 'y'])
tsne_df['words'] = list(w2v_model.wv.vocab.keys())[:10000]

# plotting. the corresponding word appears when you hover on the data point.
plot_tfidf.scatter(x='x', y='y', source=tsne_df)
hover = plot_tfidf.select(dict(type=HoverTool))
hover.tooltips={"word": "@words"}
show(plot_tfidf)

Loading BokehJS ...

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 10000 samples in 1.534s...
[t-SNE] Computed neighbors for 10000 samples in 440.930s...
[t-SNE] Computed conditional probabilities for sample 1000 / 10000
[t-SNE] Computed conditional probabilities for sample 2000 / 10000
[t-SNE] Computed conditional probabilities for sample 3000 / 10000
[t-SNE] Computed conditional probabilities for sample 4000 / 10000
[t-SNE] Computed conditional probabilities for sample 5000 / 10000
[t-SNE] Computed conditional probabilities for sample 6000 / 10000
[t-SNE] Computed conditional probabilities for sample 7000 / 10000
[t-SNE] Computed conditional probabilities for sample 8000 / 10000
[t-SNE] Computed conditional probabilities for sample 9000 / 10000
[t-SNE] Computed conditional probabilities for sample 10000 / 10000
[t-SNE] Mean sigma: 1.235973
[t-SNE] KL divergence after 250 iterations with early exaggeration: 176.225616
[t-SNE] Error after 1000 iterations: 4.608722


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
print ('building tf-idf matrix ...')
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
matrix = vectorizer.fit_transform(x_train)
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print ('vocab size :', len(tfidf))

building tf-idf matrix ...
vocab size : 65846


In [18]:
def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += w2v_model[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [19]:
from sklearn.preprocessing import scale
train_vecs_w2v = np.concatenate([buildWordVector(z, 3000) for z in tqdm(map(lambda x: x, x_train))])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([buildWordVector(z, 3000) for z in tqdm(map(lambda x: x, x_test))])
test_vecs_w2v = scale(test_vecs_w2v)

0it [00:00, ?it/s]C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
442359it [18:14, 404.30it/s]
49151it [23:04, 35.50it/s]


In [20]:
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Embedding, embeddings, merge
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=3000))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_vecs_w2v, np.array(y_train), epochs=100, verbose=2)

Epoch 1/100
 - 50s - loss: 0.3916 - acc: 0.8434
Epoch 2/100
 - 39s - loss: 0.3442 - acc: 0.8645
Epoch 3/100
 - 40s - loss: 0.3246 - acc: 0.8724
Epoch 4/100
 - 39s - loss: 0.3101 - acc: 0.8782
Epoch 5/100
 - 38s - loss: 0.2966 - acc: 0.8842
Epoch 6/100
 - 38s - loss: 0.2849 - acc: 0.8887
Epoch 7/100
 - 38s - loss: 0.2746 - acc: 0.8934
Epoch 8/100
 - 38s - loss: 0.2647 - acc: 0.8973
Epoch 9/100
 - 38s - loss: 0.2557 - acc: 0.9014
Epoch 10/100
 - 38s - loss: 0.2486 - acc: 0.9039
Epoch 11/100
 - 38s - loss: 0.2410 - acc: 0.9076
Epoch 12/100
 - 38s - loss: 0.2354 - acc: 0.9100
Epoch 13/100
 - 37s - loss: 0.2295 - acc: 0.9128
Epoch 14/100
 - 37s - loss: 0.2242 - acc: 0.9150
Epoch 15/100
 - 37s - loss: 0.2194 - acc: 0.9170
Epoch 16/100
 - 37s - loss: 0.2157 - acc: 0.9191
Epoch 17/100
 - 37s - loss: 0.2111 - acc: 0.9206
Epoch 18/100
 - 37s - loss: 0.2079 - acc: 0.9220
Epoch 19/100
 - 37s - loss: 0.2035 - acc: 0.9236
Epoch 20/100
 - 37s - loss: 0.1992 - acc: 0.9258
Epoch 21/100
 - 37s - loss: 0

In [21]:
score = model.evaluate(test_vecs_w2v, np.array(y_test), verbose=2)
print (score[1])

0.851315334361969


* 1500 : 0.835
* 3000 :  0.8513

In [26]:
model.save('./model/keras_model_size3000_window8_iter20_from_word2vec.model')